In [2]:
# вопрос - ответы на задания 3-6 нужны только для тех записей, где дан fuelLevel?

In [3]:
import pandas as pd
import numpy as np

In [4]:
DATA_PATH = 'data/vehicle_dataset_public/'
DATA_PROC_PATH = 'data/vehicle_proccessed/'

TAGS_FEATURES = ['fuelLevel', 'ingection', 'speedAndHeight', 'tachometer']
TAGS_TARGET = ['refueling2']
TAGS = TAGS_FEATURES + TAGS_TARGET

TIME_COLUMNS = ['DTIME', 'STARTDATE', 'ENDDATE']
FLOAT_COLUMNS = ['BEVALUE', 'SPEED', 'HEIGHT']

VEHICLE_IDS = [1, 3, 5, 19, 28]

In [5]:
from functools import reduce

# Словарь, состоящий из dataframe'ов для каждого признака
def dict_df_vehicle_i(i):
    dict_df_vehicle = {}
    for tag in TAGS:
        df = pd.read_csv(DATA_PATH +
                         f'vehicle{i}_{tag}_public.csv', sep=';')
        for col in FLOAT_COLUMNS:
            if col in df.columns and df[col].dtype == 'object':
                df[col] = df[col].str.replace(',','.').astype(float)
        df.rename(columns={'BEVALUE': tag}, inplace=True)

        for time_col in TIME_COLUMNS:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col])

        dict_df_vehicle[tag] = df

    return dict_df_vehicle

# Объединение всех "исходных" признаков из словаря dataframe'ов
# в один dataframe
def df_summary(dict_df, merge_how='outer'):
    df_to_merge = [dict_df[name] for name in TAGS_FEATURES]

    df_res = reduce(lambda left, right: pd.merge(left, right, how=merge_how, on='DTIME'),
                    df_to_merge)
    df_res.columns = df_res.columns.str.lower()

    return df_res

In [40]:
def timeDeltaToHours(timeDelta):
    return np.dot(timeDelta.dt.components, [24, 1, 1/60, 1/(60**2), 1/(60**3), 1/(60**4), 1/(60**5)])

In [41]:
def approximateDerivative(time, value):
    timeShifted = np.full(len(time), np.nan)
    timeShifted[1:] = time[:-1]
    timeShifted = pd.to_datetime(timeShifted)
    timeDelta = timeDeltaToHours(time - timeShifted)
    
    valueShifted = np.full(len(value), np.nan)
    valueShifted[1:] = value[:-1]
    
    valueDelta = value - valueShifted

    return valueDelta / timeDelta

In [48]:
def smooth(values):
    return pd.Series(values).rolling(10).mean()

In [65]:
vehiclesFeatures = []

for vehicle_id in VEHICLE_IDS:
    dict_df = dict_df_vehicle_i(vehicle_id)
    
    df_sum = df_summary(dict_df, merge_how='inner')
    
    df_sum['fuellevel'] = smooth(df_sum['fuellevel'])
    df_sum['speed'] = smooth(df_sum['speed'])
    df_sum['height'] = smooth(df_sum['height'])
    
    df_sum['LPH'] = approximateDerivative(df_sum['dtime'], df_sum['fuellevel'])
    
    df_sum['acceleration'] = approximateDerivative(df_sum['dtime'], df_sum['speed'])
    
    def mean_decrease_lph(x):
        x = x[1:] # первая запись за день - изменение с предыдущего дня
        x = x[x<0] # только уменьшение
        return np.mean(x)
    
    # В одном жне может быть несколько последовательностей измерений,
    # наверное, нужно это тоже будет как то учесть. Пока так.
    each_day_mean_lph = df_sum.groupby(df_sum['dtime'].dt.date)['LPH'].apply(mean_decrease_lph)
    
    # MedianDecLPM - медиана средней скорости уменьшения топлива за день ._.
    
    features = {
        'MaxFuelLevel': df_sum['fuellevel'].max(), 
        'MeanFuelLevel': df_sum['fuellevel'].mean(),
        'MedianFuelLevel': df_sum['fuellevel'].median(),
        'MaxSpeed': df_sum.loc[df_sum['speed']>0, 'speed'].max(), 
        'MeanSpeed': df_sum.loc[df_sum['speed']>0, 'speed'].mean(),
        'MedianSpeed': df_sum.loc[df_sum['speed']>0, 'speed'].median(),
        'MeanAbsAcceleration': df_sum['acceleration'].abs().mean(),
        'MedianAbsAcceleration': df_sum['acceleration'].abs().median(),
        'MeanTachometer': df_sum['tachometer'].mean(),
        'MedianTachometer': df_sum['tachometer'].median(),
        'MeanIngection': df_sum['ingection'].mean(),
        'MedianDecLPH': each_day_mean_lph.median(),
    }
    
    vehicleWiseFeatures = pd.Series(features)
    
    vehiclesFeatures.append(vehicleWiseFeatures)


In [66]:
X = pd.DataFrame(vehiclesFeatures, index=VEHICLE_IDS)
X

,MaxFuelLevel,MeanFuelLevel,MedianFuelLevel,MaxSpeed,MeanSpeed,MedianSpeed,MeanAbsAcceleration,MedianAbsAcceleration,MeanTachometer,MedianTachometer,MeanIngection,MedianDecLPH
1,59.69,38.865458,3.948000e+01,124.2,38.343104,30.0,160.861672,108.0,1807.216478,1568.0,0.984815,-12.449061
3,277.21,180.773407,1.873600e+02,93.4,19.148629,14.8,102.152539,24.0,1032.753226,950.0,0.999032,-10.028369
5,73.07,40.352908,4.063000e+01,124.6,48.209261,45.9,170.587938,120.0,1572.068366,1686.0,0.976389,-16.786343
19,95.00,29.941259,2.536000e+01,13.8,1.420739,1.2,8.038214,6.0,1126.779966,1110.0,0.991352,-8.823927
28,60.00,16.825789,1.890044e-13,115.1,36.022691,29.1,79.277601,0.0,817.308707,0.0,0.425162,-65.023800


In [ ]:
# у 1 и 5 относительно большая средняя скорость
# У 3 намного бак намного больше, чем у остальных
# у 19 очень маленькая средняя скорость 
# 28 часто стоит выключенным + с пустым баком, но потребляет много топлива
# + средний уровень топлива небольшой => наверное, на большие расстояния не ездит

In [138]:
# 28
dict_df = dict_df_vehicle_i(28)
df_sum = df_summary(dict_df, merge_how='inner')

print('Доля записей, в которых бак пустой:', (df_sum['fuellevel']==0).mean())
print('Доля записей, в которых зажигание выключено:', (df_sum['ingection']==0).mean())
print('Доля записей, в которых выполнено и то и то:', ((df_sum['ingection']==0) & (df_sum['fuellevel']==0)).mean())

Доля записей, в которых бак пустой: 0.593607579755337
Доля записей, в которых зажигание выключено: 0.5748380906692252
Доля записей, в которых выполнено и то и то: 0.5747781242504197


In [137]:
# Странно
# 1. Значит ли это, что при выключенном зажигании уровень топлива полагается = 0?
# 2. Почему #3 отличается от остальных?

for vehicle_id in VEHICLE_IDS:
    dict_df = dict_df_vehicle_i(vehicle_id)
    df_sum = df_summary(dict_df, merge_how='inner')
    
    meanFuelLevel = df_sum.loc[df_sum['ingection']==0, 'fuellevel'].mean()
    print(f'Средний уровень топлива у ТС #{vehicle_id} при выключенном зажигании: {meanFuelLevel:.2f}')

Средний уровень топлива у ТС #1 при выключенном зажигании: 0.00
Средний уровень топлива у ТС #3 при выключенном зажигании: 71.30
Средний уровень топлива у ТС #5 при выключенном зажигании: 0.05
Средний уровень топлива у ТС #19 при выключенном зажигании: 0.00
Средний уровень топлива у ТС #28 при выключенном зажигании: 0.00


In [67]:
# Стиль вождения может быть связан с
# изменениемя скорости (резкими/плавными), то есть с ускорением
# ну тут сложно, учитывая, что машины разные